In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4

cuda


In [2]:
with open("text/Higher-Education_and_Business_Standards.txt", "r", encoding = "utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', "'", '(', ')', '+', ',', '-', '.', '0', '1', '2', '4', '5', '6', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ô', 'ö', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)

In [4]:
n = int(.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

x, y = get_batch('train')
print("inputs: ")
print(x)
print("targets: ")
print(y)

tensor([24221, 24367,  9527, 36418])
inputs: 
tensor([[24, 71, 70,  1, 59, 70,  1, 59],
        [51, 70, 59, 65, 64,  8,  1, 59],
        [64, 55, 69, 69,  1, 57, 55, 64],
        [ 1, 56, 51, 53, 70, 65, 68, 69]])
targets: 
tensor([[71, 70,  1, 59, 70,  1, 59, 69],
        [70, 59, 65, 64,  8,  1, 59, 70],
        [55, 69, 69,  1, 57, 55, 64, 55],
        [56, 51, 53, 70, 65, 68, 69,  1]])


In [5]:
x = train_data[:block_size+1]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f'when input is {context} target is {target}')

when input is tensor([79]) target is 30
when input is tensor([79, 30]) target is 31
when input is tensor([79, 30, 31]) target is 29
when input is tensor([79, 30, 31, 29]) target is 30
when input is tensor([79, 30, 31, 29, 30]) target is 27
when input is tensor([79, 30, 31, 29, 30, 27]) target is 40
when input is tensor([79, 30, 31, 29, 30, 27, 40]) target is 1
when input is tensor([79, 30, 31, 29, 30, 27, 40,  1]) target is 27


In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets):
        logits = self.token_embedding_table(index)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)


        
        return logits